Dataset MNIST: картинки с рукописными цифрами.

x: 70_000 картинок размера 28x28.

y: 70_000 значений, какая цифра на картинке (0-9).

Обучаемся на 60_000 примерах, в результате можем распознать, какая цифра изображена на картинке, в т.ч. для тех 10_000 картинок, которые не входили в обучающую выборку.

Реализация методом KNN.

In [ ]:
from collections import Counter
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt
import pandas as pd
import time
# Более "низкоуровневый" подход: используем численные методы из библиотеки numpy. Будет посложнее.
import numpy as np

In [ ]:
# Для скачивания данных.
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
# Рассчитывает Эвклидовое расстояние.
def euclidean_distance(x,y):   
    return np.sqrt(np.sum((x-y)**2))

In [ ]:
# Скачиваем данные.
x, y = fetch_openml('mnist_784', version=1, return_X_y=True)

In [ ]:
# Посмотрим на данные.
# На момент написания: (70000, 784).
# Обратите внимание, что в этот датасете картинка 28x28 уже линеаризована в массив из 784 пикселей.
print('Количество картинок и их размерность: ', x.shape)
print('Количество правильных ответов к каждой картинке: ', y.shape)

# Стоит убедиться, что совпадает размер массива входных картинок и их правильных ответов.
# И "fail fast", если это не так. Не обучать классификатор на кривых данных.
assert x.shape[0] == y.shape[0], f"Размер массива входных картинок ({x.shape[0]}) и их правильных ответов ({y.shape[0]}) не совпадает."

Количество картинок и их размерность:  (70000, 784)
Количество правильных ответов к каждой картинке:  (70000,)


In [ ]:
n_train = 60_000
n_test = 10_000
# Граница (в размере элементов) между обучающей и тестовой выборкой.
split_boundary = 60_000

x_train, y_train = x[:n_train,:], y[:n_train]
x_test, y_test = x[split_boundary:split_boundary + n_test, :] , y[split_boundary:split_boundary + n_test]

In [ ]:
# Сделаем из Python массивов Pandas DataFrame для картинок.
x_train_df = pd.DataFrame(x_train)
x_test_df = pd.DataFrame(x_test)

In [ ]:
# Сделаем из Python массивов Pandas DataFrame для правильных ответов.
y_train_df = pd.DataFrame(data = y_train, columns = ['Цифра'])
y_test_df = pd.DataFrame(data = y_test, columns = ['Цифра'])

In [ ]:
# Список Эвклидовых расстояний.
train_euc_dist = []
# Список индексов
train_idxs = []
# Какие значения K пробуем.
k_vals = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]
# Список списков для хранения прогнозируемого значения для каждого K.
train_predictions = [[] for _ in range(len(k_vals))]

# Обучение.
for i in range(0, n_train):
    train_vec_1 = x_train_df.iloc[i]
    # Сравниваем каждый с каждым.
    for j in range(0, n_train):
        train_vec_2 = x_train_df.iloc[j]
        # Вычисляем Эвклидово расстояние.
        euc_dist = euclidean_distance(train_vec_1, train_vec_2)
        train_euc_dist.append(euc_dist)
        # Сохраняем индекс.
        train_idxs.append(j)

    # Словарь с результатами.
    d = {'index': train_idxs, 'distance': train_euc_dist}
    # Создаём Pandas DataFrame
    df = pd.DataFrame(d, columns=['index', 'distance'])
    # Сортируем во возрастанию Эвклидова расстояния.
    df_sorted = df.sort_values(by='distance')

    # Итерируемся по пробным значениям K.
    for k in range(len(k_vals)):
        index_list = list(df_sorted['index'][:k_vals[k]])
        distance = list(df_sorted['distance'][:k_vals[k]])
        res_list = [y_train[i] for i in index_list]
        # Ищем класс с максимальным количеством "голосов".
        pred_value = max(res_list, key=res_list.count)
        # Сохраняем прогнозируемое значение.
        train_predictions[k].append(pred_value)

    # Обнуляем списки.
    train_idxs = []
    train_euc_dist = []

In [ ]:
# Тестирование.
for i in range(0, n_test):
    test_vec_1 = df_test.iloc[i]
    # Каждый с каждым.
    for j in range(0, n_test):
        train_vec_2 = df_train.iloc[j]
        # Вычисляем Эвклидово расстояние.
        euc_dist = euclidean_distance(test_vec_1, train_vec_2)
        test_dist.append(euc_dist)
        # Сохраняем индекс.
        test_idxs.append(j)

    # Словарь с результатами.
    d = {'index': test_idxs, 'distance': test_dist}
    # Создаём Pandas DataFrame
    df = pd.DataFrame(d, columns=['index', 'distance'])
    # Сортируем во возрастанию Эвклидова расстояния.
    df_sorted = df.sort_values(by='distance')

    # Итерируемся по пробным значениям K.
    for k in range(len(k_vals)):
        index_list = list(df_sorted['index'][:k_vals[k]])
        distance = list(df_sorted['distance'][:k_vals[k]])
        res_list = [train_predictions[k][ind] for ind in index_list]
        # Ищем класс с максимальным количеством "голосов".
        pred_value = max(res_list, key=res_list.count)
        # Сохраняем прогнозируемое значение.
        test_pred_lists[k].append(pred_value)

    # Обнуляем списки.
    test_idxs = []
    test_dist = []

In [ ]:
# Вычисляем результаты для тестовой выборки.
test_pred = 0
test_pred_result = []
for K in range(len(k_values)):
    for l1,l2 in zip(test_pred_lists[K], y_test.tolist()):
        if l1 == l2:
            test_pred += 1
    accuracy = test_pred/1000
    test_pred_result.append((round(accuracy*100,2)))
    print('The test accuracy is '+str(accuracy*100)+'% for K='+str(k_values[K]))
    test_pred = 0

The test accuracy is 90.4% for K=1
The test accuracy is 90.9% for K=3
The test accuracy is 91.5% for K=5
The test accuracy is 90.4% for K=7
The test accuracy is 89.8% for K=9
The test accuracy is 88.4% for K=11
The test accuracy is 88.9% for K=13
The test accuracy is 88.3% for K=15
The test accuracy is 87.9% for K=17
The test accuracy is 87.6% for K=19
The test accuracy is 86.8% for K=21
